In [9]:
import os
from typing import Dict
import requests
from tavily import TavilyClient
from langchain_core.tools import tool

from langchain_core.messages import (
    SystemMessage,
    AIMessage,
    HumanMessage,
    ToolMessage
)

from langchain_openai import ChatOpenAI
from langgraph.graph import START, END, StateGraph
from langgraph.graph.message import MessagesState, add_messages
from langgraph.prebuilt import ToolNode
from IPython.display import Image, display

from dotenv import load_dotenv


In [10]:
load_dotenv()

True

In [11]:
@tool
def random_got_quote_tool()->Dict:
    """Randomly select a quote from the Game of Thrones series"""
    response = requests.get("https://api.gameofthronesquotes.xyz/v1/random")
    return response.json()


In [14]:
random_got_quote_tool.invoke({})

{'sentence': "Your lips are moving and you're complaining about something. That's whinging. This one's been killed six times. (points to Beric) You don't hear him bitching about it.",
 'character': {'name': 'Sandor Clegane',
  'slug': 'sandor',
  'house': {'name': "House Clegane of Clegane's Keep", 'slug': 'clegane'}}}

In [ ]:
tavily_client = TavilyClient(
    api_key=os.getenv("TAVILY_API_KEY"))